In [ ]:
#importing reuqired dependencies
from pptx import Presentation
import pandas as pd
import matplotlib.pyplot as plt
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
import os
import smtplib
import matplotlib.pyplot as plt
import numpy as np
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.message import EmailMessage
from email.mime.base import MIMEBase
from geopy import distance
pd.set_option('display.max_columns',None)
import codecs
import seaborn as sns
from email import encoders
from email.mime.application import MIMEApplication
from io import BytesIO


### Exploratory Data Analysis of above data

In [ ]:

#EDA 
df.drop([6868,6264],axis=0, inplace=True)
df['ontime/delay'] = df.ontime.replace({np.NaN, 'G'}, {0, 1})
df['vehicle_states'] = df.vehicle_no.astype(str).str[:2]
df['Origin_states'] = df['Origin_Location'].str.split(',').apply(lambda x: x[-1])
df['Dest_states'] = df['Destination_Location'].str.split(',').apply(lambda x: x[-1])
df['vehicle_states']=df['vehicle_states'].replace(('tn', 'hr'), ('TN', 'HR'))
df['Origin_states']=df['Origin_states'].replace((' Maharashtra', 'TAMIL NADU', ' Gujarat', ' Tamil Nadu',
                                                     'RAJASTHAN', ' Haryana', 'PONDICHERRY',
                                                     ' Karnataka', 'KARNATAKA', 'GUJARAT', 'HARYANA', ' Rajasthan', 
                                                     ' Uttar Pradesh', ' Pondicherry', ' West Bengal', ' Odisha',
                                                     ' Jharkhand', ' Bihar', ' Assam', ' Andhra Pradesh', ' Telangana',
                                                     ' Chattisgarh', ' Delhi', ' Kerala', ' Chandigarh', ' India', 
                                                     'UTTAR PRADESH'),
                                                    ('Maharashtra', 'Tamil Nadu', 'Gujarat', 'Tamil Nadu',
                                                     'Rajasthan', 'Haryana', 'Pondicherry',
                                                     'Karnataka', 'Karnataka', 'Gujarat', 'Haryana', 'Rajasthan',
                                                     'Uttar Pradesh', 'Pondicherry', 'West Bengal', 'Odisha',
                                                     'Jharkhand', 'Bihar', 'Assam', 'Andhra Pradesh', 'Telangana',
                                                     'Chattisgarh', 'Delhi', 'Kerala', 'Chandigarh', 'India',
                                                     'Uttar Pradesh'))
distances_km = []
for row in df.itertuples(index=False):
    distances_km.append( distance.distance(row.Org_lat_lon, row.Des_lat_lon).km
   )

df['Org_Dest_distance'] = distances_km
#Filling name "unkown" for null values in vehicle type column
df['vehicleType']=df['vehicleType'].fillna('Unknown')

#filling pervious date for actual.eta
df['actual_eta']=df['actual_eta'].fillna(method='ffill')    
#imputing transportation distence with mean value
df['TRANSPORTATION_DISTANCE_IN_KM']= df["TRANSPORTATION_DISTANCE_IN_KM"].rolling(min_periods=1, center=True, window=12).mean()


In [ ]:
def generate_report_1(row):
    
    columns = ['supplierNameCode', 'TRANSPORTATION_DISTANCE_IN_KM']
    insights_name = 'Star_Suppliers_with_More_OnTime_Deliveries'

    # Extracting relevant information from the specified columns
    relevant_data = row[columns]

    data = df[df['ontime/delay'] == 1][columns].groupby(['supplierNameCode']).agg('sum').sort_values('TRANSPORTATION_DISTANCE_IN_KM', ascending=False).head(10).reset_index()
    print(f"{insights_name}: {data['supplierNameCode'].tolist()}")

    # Bar graph
    plt.figure(figsize=(10, 6))
    plt.bar(data['supplierNameCode'], data['TRANSPORTATION_DISTANCE_IN_KM'], color='skyblue')
    plt.title('Top 10 Suppliers with the Most On-Time Deliveries')
    plt.xlabel('Supplier ID')
    plt.ylabel('Number of On-Time Deliveries')
    plt.xticks(rotation=45, ha='right')  # Rotating x-axis labels for better readability
    graph_fig = plt.gcf()

    return pd.DataFrame(data), graph_fig

def generate_report_2(row):
    
    columns = ['supplierNameCode', 'TRANSPORTATION_DISTANCE_IN_KM']
    insights_name = 'Star_Suppliers_with_More_Delayed_Deliveries'

    relevant_data = row[columns]

    data = df[df['ontime/delay'] == 0][columns].groupby(['supplierNameCode']).agg('sum').sort_values('TRANSPORTATION_DISTANCE_IN_KM', ascending=False).head(10).reset_index()
    print(f"{insights_name}: {data['supplierNameCode'].tolist()}")

    # Plot a bar graph
    plt.figure(figsize=(10, 6))
    plt.(data['supplierNameCode'], data['TRANSPORTATION_DISTANCE_IN_KM'], color='skyblue')
    plt.title('Top 10 Suppliers with the Most Delayed Deliveries')
    plt.xlabel('Supplier ID')
    plt.ylabel('Number of Delayed Deliveries')
    plt.xticks(rotation=45, ha='right')  
    graph_fig = plt.gcf()

    return pd.DataFrame(data), graph_fig
def generate_report_3(row):
    
    order=df['vehicle_states'].value_counts().head(10)
    plt.rcParams['figure.figsize']=10,5
    sns.countplot(df['vehicle_states'],
                 order=df['vehicle_states'].value_counts().head(10).index,
                  hue=df['ontime/delay'],
                 palette='magma')
    plt.xticks(rotation=90)
    plt.title('which state vehicles having most successfull ontime delivery')
    plt.legend(['Delay', 'Ontime'])
    graph_fig = plt.gcf()
    #graph = plt.show()#
    return pd.DataFrame(order),graph_fig


# Function to create a ppt presentation
def create_ppt(insights_df, insights_name, graph_fig):
    prs = Presentation()

    # Adding a title slide
    title_slide_layout = prs.slide_layouts[0]
    title_slide = prs.slides.add_slide(title_slide_layout)
    title = title_slide.shapes.title
    title.text = insights_name


    # Adding a slide with the graph
    content_slide_layout = prs.slide_layouts[5]
    content_slide = prs.slides.add_slide(content_slide_layout)

    # Saving the graph as an image
    image_stream = BytesIO()
    graph_fig.savefig(image_stream, format='png', bbox_inches='tight')
    image_stream.seek(0)
    content_slide.shapes.add_picture(image_stream, Inches(1), Inches(1), width=Inches(8), height=Inches(5))
    plt.close(graph_fig)

    # Saving the insights dataframe as a table in the presentation
    df_slide_layout = prs.slide_layouts[5]
    df_slide = prs.slides.add_slide(df_slide_layout)
    df_table = df_slide.shapes.add_table(
        insights_df.shape[0] + 1, insights_df.shape[1], Inches(1), Inches(1), width=Inches(8), height=Inches(4)
    ).table
    df_table.columns[0].width = Inches(3)
    for i, col in enumerate(insights_df.columns):
        df_table.cell(0, i).text = col
        for j, value in enumerate(insights_df[col]):
              df_table.cell(j + 1, i).text = str(value)

    # Saved the presentation
    ppt_filename = f"{insights_name}_report.pptx"
    prs.save(ppt_filename)

    return ppt_filename

# Example
data1, graph_fig1 = generate_report_1(df)
data2, graph_fig2 = generate_report_2(df)
data3, graph_fig3 = generate_report_3(df)

ppt_filename1 = create_ppt(data1, 'Name of Star_Suppliers_with_More_OnTime_Deliveries', graph_fig1)
ppt_filename2 = create_ppt(data2, 'Name of Star_Suppliers_with_More_Delayed_Deliveries', graph_fig2)
ppt_filename3 = create_ppt(data3, 'State vehicles having most successfull ontime delivery', graph_fig3)

print(f"PowerPoint file created: {ppt_filename1}")
print(f"PowerPoint file created: {ppt_filename2}")
print(f"PowerPoint file created: {ppt_filename3}")

In [ ]:
def send_email(to_email, subject, body, attachment_path):
     # Sender email configuration
    sender_email = "gmail.com"
    sender_password = ""
    smtp_server = "smtp.gmail.com"
    smtp_port = 587

    # Creating the email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject

    # Attaching the body of the email
    msg.attach(MIMEText(body, 'plain'))

    # Attaching the Ppt presentation
    with open(attachment_path, 'rb') as attachment:
        part = MIMEApplication(attachment.read(), Name="report.pptx")
    part['Content-Disposition'] = f'attachment; filename="{attachment_path}"'
    msg.attach(part)

    # Connecting to the SMTP server and send the email
    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, to_email, msg.as_string())
        print(f"Email sent to {to_email}")
    except Exception as e:
        print(f"Failed to send email to {to_email}. Error: {e}")

stakeholders = ["gmail1.com", "gmail2.com"]


for stakeholder_email  in stakeholders:
    ppt_filename3 = create_ppt(data, 'which state vehicles having most successfull ontime delivery', graph_fig3)
    send_email(stakeholder_email, 'On-Time Deliveries Report', 'Please find attached the On-Time Deliveries Report', ppt_filename3)
    ppt_filename2 = create_ppt(data, 'yes Star_Suppliers_with_More_Delayed_Deliveries', graph_fig2)
    send_email(stakeholder_email, 'Vehicles having most successfull ontime delivery', 'Please find attached the On-Time Deliveries Report', ppt_filename2)

    


In [ ]:
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

excel_file_path = "Delivery truck trip data.xlsx"
df = pd.read_excel(excel_file_path)
print(df.head())
mean_values = df.mean()

summary_df = pd.DataFrame({'Mean Values': mean_values})

summary_file_path = 'summary_file.xlsx'
summary_df.to_excel(summary_file_path)

sender_email = 'gmail.com'
sender_password = ""
recipient_emails = ['gmail1.com', 'gmail2.com']
subject = 'Insights Report'
body = 'Please find attached the insights report.'

msg = MIMEMultipart()
msg['From'] = sender_email
msg['To'] = ', '.join(recipient_emails)
msg['Subject'] = subject
msg.attach(MIMEText(body, 'plain'))

# Attaching the Excel file
with open(summary_file_path, 'rb') as file:
    attachment = MIMEApplication(file.read(), _subtype='xlsx')
    attachment.add_header('Content-Disposition', 'attachment', filename=summary_file_path)
    msg.attach(attachment)

# Connecting to the SMTP server and send the email
with smtplib.SMTP('smtp.gmail.com', 587) as server:
    server.starttls()
    server.login(sender_email, sender_password)
    server.sendmail(sender_email, recipient_emails, msg.as_string())

print('Email sent successfully.')

In [ ]:
def generate_report_11(row):
    
    columns = ['supplierNameCode', 'TRANSPORTATION_DISTANCE_IN_KM']
    insights_name = 'Star_Suppliers_with_More_OnTime_Deliveries'

    # Extracting relevant information from the specified columns
    relevant_data = row[columns]

    data = df[df['ontime/delay'] == 1][columns].groupby(['supplierNameCode']).agg('sum').sort_values('TRANSPORTATION_DISTANCE_IN_KM', ascending=False).head(10).reset_index()
    print(f"{insights_name}: {data['supplierNameCode'].tolist()}")

    # Bar graph
    plt.figure(figsize=(10, 6))
    plt.bar(data['supplierNameCode'], data['TRANSPORTATION_DISTANCE_IN_KM'], color='skyblue')
    plt.title('Top 10 Suppliers with the Most On-Time Deliveries')
    plt.xlabel('Supplier ID')
    plt.ylabel('Number of On-Time Deliveries')
    plt.xticks(rotation=45, ha='right')  # Rotating x-axis labels for better readability
    graph_fig = plt.gcf()

    return pd.DataFrame(data)